# Modèle linéaire

Considérons la cas classique d'une fonction affine :

$$y=ax+b$$

Ici, $a$ et $b$ sont des réels. Ces deux nombres définissent entièrement la courbe et permet donc d'obtenir une relation **affine** entre $x$ et $y$. En statistique, cette relation est à la base des modèles dit **linéaires**, où une variable réponse se définit comme une somme de variables explicatives où chacune de ces dernières sont multipliés par un coefficient.


## Modèle linéaire simple

![](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3a/Linear_regression.svg/438px-Linear_regression.svg.png)

Dans le modèle linéaire simple (une seule variable explicative), on suppose que la variable réponse suit le modèle suivant :

$$y_i=\beta_0 + \beta_1 x_i + \varepsilon_i$$

On remarque la ressemblance avec la fonction affine présentée ci-dessus. La différence réside dans l'existence du terme aléatoire (appelé bruit) $\varepsilon_i$. Afin de considérer le modèle, il est nécessaire de se placer sous les hypothèses suivantes

$$(\mathcal{H}) : \left\{\begin{matrix}
\mathbb{E}[\varepsilon_i]=0\\ 
\text{Cov}(\varepsilon_i, \varepsilon_j)=\delta_{ij} \sigma^2
\end{matrix}\right.$$
Les différents éléments qui interviennent sont :

- $\beta_0$ : l'ordonnée à l'origine (nommée *intercept*)
- $\beta_1$ : le coefficient directeur
- $x_i$ : l'observation $i$
- $y_i$ : le $i$-ème prix
- $\varepsilon_i$ : le bruit aléatoire liée à la $i$-ème observation

La solution peut se calculer facilement via les formules fermées suivantes :

$$\hat{\beta}_1=\frac{\sum_{i=1}^n (x_i - \bar{x})(y_i - \bar{y})}{\sum_{i=1}^n (x_i - \bar{x})^2} \qquad \hat{\beta}_0 = \hat{y} - \hat{\beta}_1 \bar{x}$$

## Modèle linéaire multiple

Dans le cas multiple (pour $p$ variables explicatives), pour la $i$-ème observation, le modèle s'écrit :

$$y_i= \beta_0 + \sum_{j=1}^p \beta_j x_{ij} + \varepsilon_i$$

Ainsi, une observation $x_i$ n'est plus une valeur, mais un **vecteur** $(x_{i1}, \dots, x_{ip})$. Il est plus commode de regrouper ces prix $y_i$ et ces vecteurs d'observations $x_i$ dans des matrices :

$$Y=X \beta + \varepsilon$$

Sous les hypothèses équivalentes du modèle simple en plus grand dimension

$$(\mathcal{H}) : \left\{\begin{matrix}
\text{rank}(X)=p\\ 
\mathbb{E}[\varepsilon]=0 \text{ et }\text{Var}(\varepsilon)=\sigma^2 I_p
\end{matrix}\right.$$

Les différents éléments qui interviennent sont :

- $\beta$ : le vecteur directeur
- $X$ : la matrice des observations
- $Y$ : le vecteur de prix
- $\varepsilon$ : le vecteur de bruit

Avec $X=( \mathbf{1}, X_1, \dots, X_n)$, $Y=(y_1, \dots, y_n)^\top$ et $\varepsilon=(\varepsilon_1, \dots, \varepsilon_n)^\top$. La solution des MCO (Moindres Carrés Ordinaires) est alors :

$$\hat{\beta}= (X^\top X)^{-1} X^\top Y$$

Vous pouvez d'ailleurs faire la démonstration de votre coté ! Pour plus d'information mathématiques, le portail de wikipédia qui est très bien fait : [lien ici](https://fr.wikipedia.org/wiki/Portail:Probabilit%C3%A9s_et_statistiques)

# Implémenter une régression linéaire 


In [1]:
#importer vos librairies 
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [2]:
#charger les données dans le drive : 2 datasets
    #price_availability.csv
    #listings_final.csv
#vérifier si tous les individus ont bien un prix 
fichierprix = pd.read_csv('price_availability.csv', sep =';')
fichierliste = pd.read_csv('listings_final.csv', sep =';')

In [3]:
fichierprix.head()

,listing_id,day,created,available,local_currency,local_price,min_nights
0,9810829,2018-12-08,2018-09-27 06:14:10.000+0000,True,EUR,160,1
1,9810829,2018-12-08,2018-09-26 19:34:02.000+0000,True,EUR,160,1
2,20897010,2018-12-09,2018-09-27 10:38:57.000+0000,True,EUR,172,2
3,20897010,2018-12-09,2018-09-27 06:10:27.000+0000,True,EUR,172,2
4,20897010,2018-12-09,2018-09-26 19:30:25.000+0000,True,EUR,172,2


In [4]:
fichierprix.shape

(4748696, 7)

In [5]:
fichierliste.head()

,Unnamed: 0,listing_id,name,type,city,neighborhood,latitude,longitude,person_capacity,beds,bedrooms,bathrooms,is_rebookable,is_new_listing,is_fully_refundable,is_host_highly_rated,is_business_travel_ready,pricing_weekly_factor,pricing_monthly_factor
0,0,28581061,La maison Clery,private_room,Paris,2e arrondissement,48.869292,2.348335,1,1,1,2.0,False,True,True,False,False,1.00,1.00
1,1,661961,studio PARIS PLACE EDITH PIAF 75020,entire_home,Paris,NaN,48.867284,2.403255,2,1,1,1.0,False,False,True,True,False,0.88,0.69
2,2,1261705,chambre privée à louer @ paris oberkampf,private_room,Paris,NaN,48.867894,2.375897,1,1,1,1.0,False,False,True,True,False,1.00,1.00
3,3,1318834,Appartement au coeur du Marais,entire_home,Paris,République,48.870370,2.358510,3,2,2,1.0,False,False,True,False,False,0.82,0.48
4,4,1677091,Lovely & Quiet flat,entire_home,Paris,Buttes-Chaumont - Belleville,48.874149,2.373700,2,1,1,1.0,False,False,True,True,False,0.95,0.90


In [6]:
fichierliste.shape

(1000, 19)

In [7]:
#vérifier si tous les individus ont bien un prix 

In [8]:
fichierprix = fichierprix.dropna(subset = ['local_price'])

In [9]:
fichierprix.head()

,listing_id,day,created,available,local_currency,local_price,min_nights
0,9810829,2018-12-08,2018-09-27 06:14:10.000+0000,True,EUR,160,1
1,9810829,2018-12-08,2018-09-26 19:34:02.000+0000,True,EUR,160,1
2,20897010,2018-12-09,2018-09-27 10:38:57.000+0000,True,EUR,172,2
3,20897010,2018-12-09,2018-09-27 06:10:27.000+0000,True,EUR,172,2
4,20897010,2018-12-09,2018-09-26 19:30:25.000+0000,True,EUR,172,2


In [10]:
fichierprix = fichierprix.dropna(subset = ['local_price'])

In [11]:
fichierliste.shape

(1000, 19)

In [12]:
fichierprix.shape

(4748696, 7)

On a bien le même nombre de lignes, cad 4748696, ce qui veut dire que tous les individus ont bien un prix

## Données d'entrée

L'objectif ici est de charger les données pour créer les matrices $X$ et $Y$ du modèle linéaire. **Attention**, il n'est pas nécessaire de rajouter le vecteur colonne $\mathbf{1}$ en première colonne, car *scikit-learn* le fait automatiquement !

In [ ]:
#définir 2 variables de travail
#X := les features à utiliser 
#Y := la target (prix)


Les informations des deux dataset sont complémentaires D'où l'intérêt de les recouper
afin de mener des analyses pertinentes.
Pour cela, nous allons utiliser la méthode merge

In [13]:
fichierbnb = pd.merge(fichierliste,fichierprix)
fichierbnb

,Unnamed: 0,listing_id,name,type,city,neighborhood,latitude,longitude,person_capacity,beds,...,is_host_highly_rated,is_business_travel_ready,pricing_weekly_factor,pricing_monthly_factor,day,created,available,local_currency,local_price,min_nights
0,0,28581061,La maison Clery,private_room,Paris,2e arrondissement,48.869292,2.348335,1,1,...,False,False,1.0,1.0,2018-10-28,2018-09-27 10:40:22.000+0000,False,EUR,47,1
1,0,28581061,La maison Clery,private_room,Paris,2e arrondissement,48.869292,2.348335,1,1,...,False,False,1.0,1.0,2018-10-28,2018-09-27 06:06:07.000+0000,False,EUR,47,1
2,0,28581061,La maison Clery,private_room,Paris,2e arrondissement,48.869292,2.348335,1,1,...,False,False,1.0,1.0,2018-10-28,2018-09-26 19:31:53.000+0000,False,EUR,47,1
3,0,28581061,La maison Clery,private_room,Paris,2e arrondissement,48.869292,2.348335,1,1,...,False,False,1.0,1.0,2018-09-01,2018-09-27 10:40:22.000+0000,True,EUR,47,1
4,0,28581061,La maison Clery,private_room,Paris,2e arrondissement,48.869292,2.348335,1,1,...,False,False,1.0,1.0,2018-09-01,2018-09-27 06:06:07.000+0000,True,EUR,47,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
402611,999,28682896,Village Jourdain,private_room,Paris,Buttes-Chaumont - Belleville,48.875425,2.395240,2,1,...,False,False,1.0,1.0,2018-10-16,2018-09-27 06:16:22.000+0000,True,EUR,40,1
402612,999,28682896,Village Jourdain,private_room,Paris,Buttes-Chaumont - Belleville,48.875425,2.395240,2,1,...,False,False,1.0,1.0,2018-10-16,2018-09-26 19:36:28.000+0000,True,EUR,39,1
402613,999,28682896,Village Jourdain,private_room,Paris,Buttes-Chaumont - Belleville,48.875425,2.395240,2,1,...,False,False,1.0,1.0,2018-09-10,2018-09-27 10:50:12.000+0000,True,EUR,39,1
402614,999,28682896,Village Jourdain,private_room,Paris,Buttes-Chaumont - Belleville,48.875425,2.395240,2,1,...,False,False,1.0,1.0,2018-09-10,2018-09-27 06:16:22.000+0000,True,EUR,39,1


In [14]:
fichierbnb.columns

Index(['Unnamed: 0', 'listing_id', 'name', 'type', 'city', 'neighborhood',
       'latitude', 'longitude', 'person_capacity', 'beds', 'bedrooms',
       'bathrooms', 'is_rebookable', 'is_new_listing', 'is_fully_refundable',
       'is_host_highly_rated', 'is_business_travel_ready',
       'pricing_weekly_factor', 'pricing_monthly_factor', 'day', 'created',
       'available', 'local_currency', 'local_price', 'min_nights'],
      dtype='object')

In [15]:
fichierbnb.shape

(402616, 25)

On a bien le même nombre de lignes, cad 4748696, ce qui veut dire que tous les individus ont bien un prix

In [ ]:
#définir 2 variables de travail
#X := les features à utiliser 
#Y := la target (prix)

Toute forme de business a besoin de visibilité financière sur toutes les échelles du temps
D'où l'importance de minimiser les risques d'erreur dans les prédictions
Ce qui nous oblige à sélectionner les données dites pertinentes

In [16]:
from sklearn.model_selection import train_test_split
#définition des features et de la target
X = ['listing_id', 'beds', 'pricing_weekly_factor', 'available', 'min_nights']
Y = ['local_price']

In [33]:
available_label_encoder = LabelEncoder()
fichierbnb.available=available_label_encoder.fit_transform(fichierbnb.available)
fichierbnb

,Unnamed: 0,listing_id,name,type,city,neighborhood,latitude,longitude,person_capacity,beds,...,is_host_highly_rated,is_business_travel_ready,pricing_weekly_factor,pricing_monthly_factor,day,created,available,local_currency,local_price,min_nights
0,0,28581061,La maison Clery,private_room,Paris,2e arrondissement,48.869292,2.348335,1,1,...,False,False,1.0,1.0,2018-10-28,2018-09-27 10:40:22.000+0000,0,EUR,47,1
1,0,28581061,La maison Clery,private_room,Paris,2e arrondissement,48.869292,2.348335,1,1,...,False,False,1.0,1.0,2018-10-28,2018-09-27 06:06:07.000+0000,0,EUR,47,1
2,0,28581061,La maison Clery,private_room,Paris,2e arrondissement,48.869292,2.348335,1,1,...,False,False,1.0,1.0,2018-10-28,2018-09-26 19:31:53.000+0000,0,EUR,47,1
3,0,28581061,La maison Clery,private_room,Paris,2e arrondissement,48.869292,2.348335,1,1,...,False,False,1.0,1.0,2018-09-01,2018-09-27 10:40:22.000+0000,1,EUR,47,1
4,0,28581061,La maison Clery,private_room,Paris,2e arrondissement,48.869292,2.348335,1,1,...,False,False,1.0,1.0,2018-09-01,2018-09-27 06:06:07.000+0000,1,EUR,47,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
402611,999,28682896,Village Jourdain,private_room,Paris,Buttes-Chaumont - Belleville,48.875425,2.395240,2,1,...,False,False,1.0,1.0,2018-10-16,2018-09-27 06:16:22.000+0000,1,EUR,40,1
402612,999,28682896,Village Jourdain,private_room,Paris,Buttes-Chaumont - Belleville,48.875425,2.395240,2,1,...,False,False,1.0,1.0,2018-10-16,2018-09-26 19:36:28.000+0000,1,EUR,39,1
402613,999,28682896,Village Jourdain,private_room,Paris,Buttes-Chaumont - Belleville,48.875425,2.395240,2,1,...,False,False,1.0,1.0,2018-09-10,2018-09-27 10:50:12.000+0000,1,EUR,39,1
402614,999,28682896,Village Jourdain,private_room,Paris,Buttes-Chaumont - Belleville,48.875425,2.395240,2,1,...,False,False,1.0,1.0,2018-09-10,2018-09-27 06:16:22.000+0000,1,EUR,39,1


In [34]:
fichierbnb[["available"]] = fichierbnb[["available"]].apply(pd.to_numeric)

In [35]:
fichierbnb.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 402616 entries, 0 to 402615
Data columns (total 25 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   Unnamed: 0                402616 non-null  int64  
 1   listing_id                402616 non-null  int64  
 2   name                      402616 non-null  object 
 3   type                      402616 non-null  object 
 4   city                      402616 non-null  object 
 5   neighborhood              377260 non-null  object 
 6   latitude                  402616 non-null  float64
 7   longitude                 402616 non-null  float64
 8   person_capacity           402616 non-null  int64  
 9   beds                      402616 non-null  int64  
 10  bedrooms                  402616 non-null  int64  
 11  bathrooms                 402616 non-null  float64
 12  is_rebookable             402616 non-null  bool   
 13  is_new_listing            402616 non-null  b

In [ ]:
#construire l'ensemble de donnée prix 
#
#    INDICE 
# 
# récupérer les prix des ID dans le dataset de prix 
# 🚧 il y a plusieurs prix dans le dataset 🚧



In [17]:
fichierbnb.listing_id.value_counts()

23566040    526
10744247    525
28709644    525
27030906    524
12976143    524
           ... 
25702020    133
25766867    133
26556587    133
24248060    132
14386985    131
Name: listing_id, Length: 999, dtype: int64

In [18]:
fichierbnb.listing_id.describe()

count    4.026160e+05
mean     1.531675e+07
std      9.280968e+06
min      5.609300e+04
25%      6.278981e+06
50%      1.703526e+07
75%      2.364286e+07
max      2.879280e+07
Name: listing_id, dtype: float64

In [19]:
fichierbnb.local_price.value_counts()

250     9038
180     8128
160     7741
40      7719
170     7269
        ... 
1144       1
2720       1
573        1
1053       1
2538       1
Name: local_price, Length: 774, dtype: int64

In [20]:
fichierbnb.local_price.describe()

count    402616.000000
mean        194.766075
std         191.812037
min           9.000000
25%          78.000000
50%         145.000000
75%         240.000000
max        3302.000000
Name: local_price, dtype: float64

In [21]:
fichierbnb2 = fichierbnb.groupby('name')

In [22]:
fichierbnb2.head(30)

,Unnamed: 0,listing_id,name,type,city,neighborhood,latitude,longitude,person_capacity,beds,...,is_host_highly_rated,is_business_travel_ready,pricing_weekly_factor,pricing_monthly_factor,day,created,available,local_currency,local_price,min_nights
0,0,28581061,La maison Clery,private_room,Paris,2e arrondissement,48.869292,2.348335,1,1,...,False,False,1.0,1.0,2018-10-28,2018-09-27 10:40:22.000+0000,False,EUR,47,1
1,0,28581061,La maison Clery,private_room,Paris,2e arrondissement,48.869292,2.348335,1,1,...,False,False,1.0,1.0,2018-10-28,2018-09-27 06:06:07.000+0000,False,EUR,47,1
2,0,28581061,La maison Clery,private_room,Paris,2e arrondissement,48.869292,2.348335,1,1,...,False,False,1.0,1.0,2018-10-28,2018-09-26 19:31:53.000+0000,False,EUR,47,1
3,0,28581061,La maison Clery,private_room,Paris,2e arrondissement,48.869292,2.348335,1,1,...,False,False,1.0,1.0,2018-09-01,2018-09-27 10:40:22.000+0000,True,EUR,47,1
4,0,28581061,La maison Clery,private_room,Paris,2e arrondissement,48.869292,2.348335,1,1,...,False,False,1.0,1.0,2018-09-01,2018-09-27 06:06:07.000+0000,True,EUR,47,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
402260,999,28682896,Village Jourdain,private_room,Paris,Buttes-Chaumont - Belleville,48.875425,2.395240,2,1,...,False,False,1.0,1.0,2018-09-16,2018-09-27 06:16:22.000+0000,True,EUR,39,1
402261,999,28682896,Village Jourdain,private_room,Paris,Buttes-Chaumont - Belleville,48.875425,2.395240,2,1,...,False,False,1.0,1.0,2018-09-16,2018-09-26 19:36:28.000+0000,True,EUR,39,1
402262,999,28682896,Village Jourdain,private_room,Paris,Buttes-Chaumont - Belleville,48.875425,2.395240,2,1,...,False,False,1.0,1.0,2018-12-17,2018-09-27 10:50:12.000+0000,True,EUR,39,1
402263,999,28682896,Village Jourdain,private_room,Paris,Buttes-Chaumont - Belleville,48.875425,2.395240,2,1,...,False,False,1.0,1.0,2018-12-17,2018-09-27 06:16:22.000+0000,True,EUR,39,1


In [23]:
fichierbnb.name.value_counts()

studette                                                       904
Hotel 4* Suite deluxe avec jacuzzi et petit déjeuner offert    526
(six bed room )I                                               525
LORD BYRON-SPACE& STYLE IN 8TH EME                             525
Vue sur Tour Eiffel Sacré-coeur #1                             524
                                                              ... 
Appartement de charme dans le Marais                           133
Light and cozy apartment at Trocadero                          133
Paris-Montmartre Charming & Cosy Appartment                    133
Charmant studio calme dans un quartier très vivant             132
Chambre avec salle de douche privative                         131
Name: name, Length: 998, dtype: int64

En *Machine Learning*, on a l'habitude de couper l'ensemble de données en deux sous-ensembles :

- Un ensemble d'entraînement (*train set*), sur lequel le modèle va être calibré.
- Un ensemble de test (*test set*), qui ne sera pas utilisé pendant le calibrage mais permettra de vérifier l'aptitude du modèle à généraliser sur de nouvelles observations inconnues.

En général, on découpe l'ensemble de données (*split*) en prenant $\alpha \%$ de l'ensemble pour entraînement et $1-\alpha \%$ comme test. Dans la plus part des cas, on considère que $\alpha=10,20 ou 30\%$.

In [ ]:
#utiliser la méthode split de sklearn en splitant avec un alpha=30 et un random state=42 
#zafficher la shape de vos données 


In [26]:
from sklearn.model_selection import train_test_split
#définition des features et de la target
features = ['listing_id', 'beds', 'pricing_weekly_factor', 'available', 'min_nights']
target = ['local_price']
#split 
x_train, x_test, y_train, y_test = train_test_split(fichierbnb[features],fichierbnb[target],test_size=0.3)

In [27]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape


((281831, 5), (120785, 5), (281831, 1), (120785, 1))

In [28]:
x_train.head(), x_test.head()

(        listing_id  beds  pricing_weekly_factor  available  min_nights
 81076      3430844    10                   0.96      False           2
 279454    21868898     1                   0.90      False           2
 220623    17973038     0                   1.00       True           3
 212913    17432320     3                   1.00       True           7
 161509    12816341     2                   1.00      False           4,
         listing_id  beds  pricing_weekly_factor  available  min_nights
 362631    26656283     2                    1.0       True           7
 375651    27147284     1                    1.0      False           1
 132171     9934524     3                    0.9       True           2
 162832    12961597     3                    1.0      False           5
 308634    23700276     1                    1.0      False           1)

In [ ]:
#créons le modele de random forest 
rf = RandomForestRegressor(random_state = 42)
#random search of parameters, en utilisant 3 fold cross validation 
#recherche dans 100 differentes combinaisons et utilisation de tout les core du processeur 
rf_ex = RandomizedSearchCV(estimator=rf, param_distributions=random_grid,
                              n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs=-1,
                              return_train_score=True)

#fit du modele 
target = np.array(y_train)
rf_ex.fit(x_train.values,target.ravel());

## Entraînement

Pour information, *scikit-learn* utilise le solveur OLS (Ordinary Least Squares) de *numpy*.

In [ ]:
#créer l'objet de régression et entrainer le sur notre ensemble d'entraînement


On affiche le vecteur des coefficients pour interpréter rapidement le modèle.

In [ ]:
#afficher les coefficients
#que remarquez vous ? 


## Validation du modèle

### Le coefficient de détermination $R^2$

Par la suite, nous ferons l'hypothèse de gaussianité sur les bruits. Dans l'idée, nous aimerions obtenir une valeur numérique qui nous indique à quel point la régression linéaire a un sens sur nos données. Pour cela, introduisons les notations suivantes :

- $SCT=\|Y-\hat{y} \mathbf{1}\|^2$ est la somme des carrés totaux
- $SCE=\|\hat{Y}-\hat{y} \mathbf{1}\|^2$ est la somme des carrés expliqués
- $SCR=\|\hat{\varepsilon}\|^2$ est la somme des carrés résiduels

L'idée est de décomposer la somme des carrés totaux comme la somme des carrés que le modèle explique, en plus de la somme des carrés qui sont liés aux résidus (et donc que le modèle ne peut pas expliquer). On voit donc ici l'intérêt de calculer un coefficient à partir du $SCE$. Puisque l'on a la relation suivante :

$$SCT=SCE+SCR \text{ alors } 1=\frac{SCE}{SCT}+\frac{SCR}{SCT}$$

Plus les résidus sont petits (et donc la régression est "bonne"), plus $SCR$ devient petit et donc $SCE$ devient grand. Le schéma inverse s'opère de la même façon. Dans le meilleur des cas, on obtient $SCR=0$ et donc $SCE=SCT$ d'où le premier membre vaut $1$. Dans le cas contraite, $SCE=0$ et automatiquement, le premier membre est nul. C'est ainsi que l'on définit le coefficient de détermination $R^2$ comme 
$$R^2=\frac{SCE}{SCT}=1-\frac{SCR}{SCT}$$
Ainsi, $R^2 \in [0,1]$. Plus $R^2$ est proche de $1$, plus la régression linéaire a du sens. Au contraire, si $R^2$ est proche de $0$, le modèle linéaire possède un faible pouvoir explicatif.

In [ ]:
#faire une prediction sur X


In [ ]:
#afficher l'erreur des moindres carrées sur l'ensemble d'entrainement ainsi que le R2


## Bonus : Analyse de l'homoscédasticité

L'analyse de l'homoscédasticité est primordiale : c'est en particulier elle qui nous permet de vérifier, à partir des résidus, si les bruits vérifient bien l'hypothèse $(\mathcal{H})$. On calcule donc les **résidus studentisés**.

$$t_i^*=\frac{\hat{\varepsilon}_i}{\hat{\sigma}_{(i)} \sqrt{1-h_{ii}}}$$
Avec $h_{ii}=\{X(X^\top X)^{-1} X^\top\}_{ii}=H_{ii}$ la matrice de projection sur l'hyperplan des variables. Plus précisément, $H$ est la matrice qui projette $Y$ sur l'espace engendré par les variables, soit $\hat{Y}=HY$. De même, on considère $\hat{\sigma}_{(i)}$ l'estimateur de la variance du bruit en supprimant l'observation $i$ (par une méthode de validation croisée Leave-One-Out que nous ne détaillerons pas ici).

Dans ce cas, on peut montrer que les résidus studentisés suivent une loi de Student à $n-p-1$ degrés de liberté.

In [ ]:
#analyser le code ci-dessous 
import scipy
Y_pred = regr.predict(X_train)
n = X_train.shape[0]
p = 4
residuals = np.abs(y_train - Y_pred)
H = np.matmul(X_train, np.linalg.solve(np.dot(X_train.T, X_train), X_train.T))
std_hat = np.dot(residuals, residuals) / (n - p)
standart_residuals = np.asarray([residuals[i] / np.sqrt(std_hat * (1 - H[i, i])) for i in range(len(residuals))])
student_residuals = np.asarray([ standart_residuals[i] * np.sqrt((n - p - 1) / (n - p - standart_residuals[i]**2)) for i in range(n) ])
cook = np.asarray([ H[i, i] * student_residuals[i] / (X_train.shape[1] * (1 - H[i, i])) for i in range(n) ])

plt.figure(figsize=(20, 12))
plt.subplot(221)
plt.scatter(Y_pred, student_residuals, s=12, c="white", edgecolors="blue")
plt.plot([min(Y_pred), max(Y_pred)], [ scipy.stats.t.ppf(q=0.975, df=n-p-1), scipy.stats.t.ppf(q=0.975, df=n-p-1)], color="green", alpha=0.6, label="Quantile de Student")
plt.title("Analyse de l’homoscédasticité")
plt.xlabel("Prédictions $\hat{y}_i$")
plt.ylabel("Résidus studentisés $|t_i^*|$")
plt.legend()